# **Gradio:** Graphs

### Show Gradio BarGraph

In [ ]:
import gradio as gr

def agent_function(agent):
    agent_data = BSA_results[BSA_results["Agent Name"]==agent]
    report = agent_data["Agent Report"].values[0]
    guidelines = agent_data["Guideline/Tips"].values[0]
    calls = agent_data["Total Calls"].values[0]
    sales = agent_data["Total Sales"].values[0]
    state = agent_data["Agent State"].values[0]
    agent_features = eval(agent_data['Agent (Average) LIME Features'].reset_index(drop=True)[0])
    top_agent_features = eval(agent_data['Top Agents (Average) LIME Features'].reset_index(drop=True)[0])
    simple = pd.DataFrame(
    {
        "Features": agent_features.keys(),
        "Agent": agent_features.values(),
        "Top_Agents": top_agent_features.values(),
    })


    return calls, sales, state, report, guidelines, simple

simple = pd.DataFrame(
{
    "Features": agent_features.keys(),
    "Agent": agent_features.values(),
    "Top_Agents": top_agent_features.values(),
})


with gr.Blocks() as demo:
    gr.Markdown("## Agent Reporting Dashboard")
    with gr.Row():
        inp = [gr.Dropdown(BSA_results["Agent Name"].values.tolist(), label="Select Agent")]
        out1 = [gr.Text(label="Total Calls"),
              gr.Text(label="Total Sales"),
              gr.Text(label="Agent State")]
    with gr.Row():
        out2 = [gr.Text(label="Agent Performance Report"),
              gr.Text(label="Training Guidelines/Tips")]
    with gr.Row():
        out3 = [gr.BarPlot(
                        simple,
                        x="Features",
                        y="Agent",
                        title="Agent Vs Top Agents (Average)",
                        tooltip=["Features", "Agent"],
                        y_lim=[0, 100],
                        )]
        
    btn = gr.Button("Run")
    btn.click(fn=agent_function, inputs=inp, outputs=(out1+out2+out3))

demo.launch()

### Show MatplotLib Graph on Gradio

In [ ]:
import gradio as gr

import matplotlib.pyplot as plt

def agent_comparison_chart(agent_name, data):
    # Find the row index where 'Agent Name' matches the provided agent_name
    index = data['Agent Name'].index[data['Agent Name'] == agent_name].tolist()
    if not index:
        print(f"Agent {agent_name} not found in the dataset.")
        return

    # Extract emotions and values for the specified agent and top agents
    agent_features = eval(data['Agent (Average) LIME Features'][index[0]])
    top_agent_features = eval(data['Top Agents (Average) LIME Features'][index[0]])

    emotions = list(agent_features.keys())
    agent_values = list(agent_features.values())
    top_agent_values = list(top_agent_features.values())

    bar_width = 0.4
    positions = range(len(emotions))

    plt.figure(figsize=(12, 6))
    agent_bars = plt.bar(positions, agent_values, bar_width, label=f'Agent {agent_name} (Average) LIME Features')
    top_agent_bars = plt.bar([pos + bar_width for pos in positions], top_agent_values, bar_width, label='Top Agents (Average) LIME Features')
    plt.xlabel('Emotions')
    plt.ylabel('Values')
    plt.title(f'Comparison of Emotion Values between Agent {agent_name} and Top Agents')
    plt.xticks([pos + bar_width / 2 for pos in positions], emotions, rotation=90)
    plt.legend()

    # Display values on top of the bars
    for bar, value in zip(agent_bars, agent_values):
        plt.text(bar.get_x() + bar.get_width() / 2, value, str(int(value)), ha='center', va='bottom')
    for bar, value in zip(top_agent_bars, top_agent_values):
        plt.text(bar.get_x() + bar.get_width() / 2, value, str(int(value)), ha='center', va='bottom')

    return plt


def agent_function(agent):
    agent_data = BSA_results[BSA_results["Agent Name"]==agent]
    report = agent_data["Agent Report"].values[0]
    guidelines = agent_data["Guideline/Tips"].values[0]
    calls = agent_data["Total Calls"].values[0]
    sales = agent_data["Total Sales"].values[0]
    state = agent_data["Agent State"].values[0]
    return calls, sales, state, report, guidelines, agent_comparison_chart(agent, BSA_results)

with gr.Blocks() as demo:
    gr.Markdown("## Agent Reporting Dashboard")
    with gr.Row():
        inp = [gr.Dropdown(BSA_results["Agent Name"].values.tolist(), label="Select Agent")]
        out1 = [gr.Text(label="Total Calls"),
              gr.Text(label="Total Sales"),
              gr.Text(label="Agent State")]
    with gr.Row():
        out2 = [gr.Text(label="Agent Performance Report"),
              gr.Text(label="Training Guidelines/Tips")]
    with gr.Row():
        out3 = [gr.Plot(scale=2)]
        
    btn = gr.Button("Run")
    btn.click(fn=agent_function, inputs=inp, outputs=(out1+out2+out3))

demo.launch()


# 'Yono Noyman'
# Aditya Chawla